In [48]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import normalize
from numpy.linalg import norm

In [78]:
from sklearn.preprocessing import normalize
df = pd.read_csv('drive/MyDrive/Quran_en_vector.csv')
df2 = pd.read_csv('drive/MyDrive/Quran_fa_vector.csv')


en = df.drop(columns=['sura', 'aya']) 
en = en.drop(df.columns[[0]], axis=1) 
norm_en = normalize(en.to_numpy(), axis=1, norm='l2')
en = torch.from_numpy(norm_en)


fa = df2.drop(columns=['sura', 'aya']) 
fa = fa.drop(df.columns[[0]], axis=1) 
norm_fa = normalize(fa[:6179].to_numpy(), axis=1, norm='l2')
fa = torch.from_numpy(norm_fa)


res = torch.cat((fa, en), 1)
df3 = pd.DataFrame(res.numpy())
df3 = df3.sample(frac = 1)

train_idx = df3.iloc[:4000, :].index
test_idx = df3.iloc[4000:, :].index

df = torch.from_numpy(df3.iloc[:6179, :100].to_numpy())
df2 = torch.from_numpy(df3.iloc[:6179, 100:].to_numpy())

print(df.shape)
print(df2.shape)


print(test_idx.shape)
print(train_idx.shape)


df3

torch.Size([6179, 100])
torch.Size([6179, 100])
(2179,)
(4000,)


,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
2210,-0.205901,0.036869,0.001264,0.091251,-0.150725,-0.295735,-0.141022,-0.098762,-0.062705,0.053448,...,-0.210672,0.165244,0.003229,-0.044015,-0.042817,-0.144846,0.123361,-0.069751,-0.077983,0.016086
4054,-0.034810,0.063046,0.021897,-0.051998,-0.211121,-0.186375,-0.061055,0.007553,-0.150457,0.110720,...,-0.150239,0.169435,-0.033161,0.044440,0.002523,-0.063308,0.098999,0.042465,-0.100717,0.018905
4613,-0.061413,0.106531,0.057097,-0.049796,-0.156797,-0.161627,-0.210342,-0.062810,0.058953,0.046729,...,-0.207197,0.160208,0.045032,-0.116909,0.024774,-0.069318,0.134121,0.050926,-0.061681,0.053943
4123,-0.009890,0.133311,-0.032778,0.129243,-0.147435,-0.159366,-0.209127,-0.146877,0.023653,0.006258,...,-0.113046,0.112698,0.042388,-0.161382,-0.060210,-0.039122,0.083270,0.012588,0.082571,0.036418
5688,-0.091512,0.174501,0.151028,0.057800,-0.076134,-0.144838,-0.056470,0.073846,-0.043226,-0.029233,...,-0.052346,0.201595,-0.049282,-0.025739,-0.082791,-0.066521,0.273285,-0.009619,0.021029,0.048850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3569,-0.048305,0.094280,0.060288,-0.082803,-0.241375,-0.185531,-0.022834,-0.120245,-0.046090,0.099896,...,-0.234457,0.025285,-0.004297,-0.058207,0.010525,-0.020168,0.066098,0.127150,-0.111956,-0.137900
1777,-0.081545,0.070733,-0.024426,-0.007243,-0.202424,-0.086647,-0.063410,-0.097910,0.028380,0.074399,...,-0.205006,0.193257,-0.018265,-0.148825,-0.097468,-0.089339,0.154412,-0.005341,-0.069451,-0.009856
2694,-0.117702,-0.148515,0.034753,-0.071181,-0.100111,-0.303397,-0.200392,-0.109256,0.049591,0.022278,...,-0.184580,0.232230,0.095502,-0.102055,0.084507,0.008446,0.145381,-0.028366,-0.085066,0.055087
2795,-0.069714,0.129192,0.107760,-0.124007,-0.221928,-0.166834,-0.050509,-0.185455,-0.057119,0.114689,...,-0.208015,0.072724,0.070264,-0.136075,-0.040668,-0.023711,0.022408,0.037355,0.034249,0.030462


In [52]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)  

    def forward(self, x):
        out = self.linear(x)
        return out

In [ ]:
from torch.nn.functional import normalize
from torch.optim.lr_scheduler import StepLR
#from sklearn.preprocessing import normalize

input_dim = 100
output_dim = 100

model = LinearRegressionModel(input_dim, output_dim)

criterion = nn.MSELoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
scheduler = StepLR(optimizer, step_size=1000, gamma=0.1)


epochs = 100000

for epoch in range(epochs):
    epoch += 1
    # Convert numpy array to torch Variable
    
    inputs = fa.requires_grad_()
    labels = en

    # Clear gradients w.r.t. parameters
    optimizer.zero_grad() 
    model.double()

    # Forward to get output
    #outputs = torch.from_numpy(normalize(model(inputs).detach().numpy(), axis=1, norm='l2'))  
    outputs = model(inputs)
    #outputs = normalize(model(inputs))

    #print(torch.norm(outputs))
    #print(np.linalg.norm(outputs[0, :]))
    #loss.requres_grad = True

    #A = outputs[0, :].detach().numpy()
    #B = labels[0, :].detach().numpy()
    #print("epoche : " + str(epoch) + "     " + str(abs(np.dot(A,B)/(norm(A)*norm(B)))))


    # Calculate Loss
    loss = criterion(outputs, labels)

    # Getting gradients w.r.t. parameters
    loss.backward()

    # Updating parameters
    optimizer.step()
 
    print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 1, loss 0.016985241508464734
epoch 2, loss 0.016905021807043903
epoch 3, loss 0.016753585221686712
epoch 4, loss 0.016539828417994378
epoch 5, loss 0.01627237967006974
epoch 6, loss 0.01595947799732259
epoch 7, loss 0.015608886589301306
epoch 8, loss 0.015227834424833064
epoch 9, loss 0.014822980865639962
epoch 10, loss 0.014400398770549072
epoch 11, loss 0.013965572346107614
epoch 12, loss 0.013523406533436052
epoch 13, loss 0.013078245239076818
epoch 14, loss 0.012633896158071254
epoch 15, loss 0.01219366031831156
epoch 16, loss 0.011760364803374938
epoch 17, loss 0.011336397392841374
epoch 18, loss 0.010923742100157475
epoch 19, loss 0.010524014793472246
epoch 20, loss 0.010138498259029032
epoch 21, loss 0.00976817621365144
epoch 22, loss 0.009413765896163925
epoch 23, loss 0.009075748970390965
epoch 24, loss 0.008754400557470387
epoch 25, loss 0.008449816285057046
epoch 26, loss 0.008161937297750053
epoch 27, loss 0.007890573218653761
epoch 28, loss 0.007635423088048745
epoch

In [56]:
torch.save(model, '/content/drive/MyDrive/transform.pt')

In [ ]:
model = torch.load('/content/drive/MyDrive/transform.pt', map_location=torch.device('cpu'))

In [74]:
from sklearn.preprocessing import normalize
A = normalize(model(en).detach().numpy(), axis=1, norm='l2')
outs = []
for i in range(30) : 
 
  A = model(fa[1, :]).detach().numpy()
  B = en[i, :]
  outs.append(abs(np.dot(A,B)/(norm(A)*norm(B))))
  #print("similarity of aya 1 in english with aya " + str(i) + " in arabic : " + str(np.dot(A,B)/(norm(A)*norm(B))))


In [75]:
print(outs)
print(max(outs))
print(outs.index(max(outs)))

[0.6985394303041478, 0.782071121643543, 0.5846369626091771, 0.7193111000426664, 0.7789317968978888, 0.7757841761132686, 0.7917870772154106, 0.44405662810296836, 0.8141950621488389, 0.8092754957186024, 0.8895614524576457, 0.7863448351114847, 0.8746707552126246, 0.794198299828259, 0.8532129959583191, 0.8172027972367706, 0.8241001231970464, 0.7981137702869759, 0.7419643854042833, 0.854498551755171, 0.8383655421664865, 0.8510176493981605, 0.8780679695885903, 0.8883732899676323, 0.7215008562867606, 0.8726145591909286, 0.8694337588763712, 0.8448020483743612, 0.8937546166500319, 0.8688371718614881]
0.8937546166500319
28
